In [2]:
import os
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind, mannwhitneyu

osl = os.listdir
ospj = os.path.join

### get relevant runs data given filters of interest

In [3]:
import re

def natural_sort(l):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key=alphanum_key)

In [4]:
def augment_expstring(base_str, hg=False, hgr=False):
    noopt_str = "NOOPT"
    opt_str = "OPT-"
    hg_str = "HG"
    hgr_str = "HGR"
    if not hg and not hgr:
        return base_str + noopt_str
    if hg and not hgr:
        return base_str + opt_str + hg_str
    if hgr and not hg:
        return base_str + opt_str + hgr_str
    if hg and hgr:
        return base_str + opt_str + hg_str + "-" + hgr_str
    raise Exception("")

def baseline_exp(exp_qualifier = "210h_10hp", hg=False, hgr=False):
    base_fcscout_str = f"fcscout_{exp_qualifier}_"
    return augment_expstring(base_fcscout_str, hg=hg, hgr=hgr) + "_"

def gcn_exp(exp_qualifier = "210h_10hp", hg=False, hgr=False, is_global=False):
    base = f"gcnscout_{exp_qualifier}_"
    return augment_expstring(base, hg=hg, hgr=hgr) + ("_global" if is_global else "_local")

def gat_exp(exp_qualifier = "210h_10hp", hg=False, hgr=False, is_global=False):
    base = f"gatscout_{exp_qualifier}_"
    return augment_expstring(base, hg=hg, hgr=hgr) + ("_global" if is_global else "_local")

def gt_exp(exp_qualifier = "210h_10hp", hg=False, hgr=False, is_global=False):
    base = f"gtscout_{exp_qualifier}_"
    return augment_expstring(base, hg=hg, hgr=hgr) + ("_global" if is_global else "_local")

def get_exps(exp_qualifier="210h_10hp", baseline_exp_string="210h_10hp", hg=False, hgr=False, is_global=False):
    return [
        baseline_exp(exp_qualifier=baseline_exp_string, hg=hg, hgr=hgr),
        gcn_exp(exp_qualifier=exp_qualifier, hg=hg, hgr=hgr, is_global=is_global),
        gat_exp(exp_qualifier=exp_qualifier, hg=hg, hgr=hgr, is_global=is_global),
        gt_exp(exp_qualifier=exp_qualifier, hg=hg, hgr=hgr, is_global=is_global),
    ]


hg=False
hgr=False
is_global=True
runs_filters = get_exps(
    exp_qualifier="210h-10hp-400bs-1em4lr-eval",
    baseline_exp_string="210h-10hp-400bs-1em4lr-eval",
    is_global=is_global,
    hg=hg,
    hgr=hgr,
)
# runs_filters = [
#     baseline_exp("210h-10hp-400bs-1em4lr-eval", hg=hg, hgr=hgr),
#     gcn_exp("210h-10hp-400bs-1em4lr-eval", hg=hg, hgr=hgr, is_global=is_global),
# ]
print(runs_filters)

maindir = "/home/vchad/ray_results/"
dir_baselines = "/home/vchad/results_aaai/final/fcscout_210h-10hp-400bs-1em4lr/"
dir_gcn_global = "/home/vchad/results_aaai/final/gcnscout_210h-10hp-400bs-1em4lr_GLOBAL/"
dir_gat_global = "/home/vchad/results_aaai/final/gatscout_210h-10hp-400bs-1em4lr_GLOBAL/"
dir_gt_global = "/home/vchad/results_aaai/final/gtscout_210h-10hp-400bs-1em4lr_GLOBAL/"
dir_gcn_local = "/home/vchad/results_aaai/final/gcnscout_210h-10hp-400bs-1em4lr_LOCAL/"
dir_gat_local = "/home/vchad/results_aaai/final/gatscout_210h-10hp-400bs-1em4lr_LOCAL/"
dir_gt_local = "/home/vchad/results_aaai/final/gtscout_210h-10hp-400bs-1em4lr_LOCAL/"
#runs_dirs = [dir_baselines, dir_gcn_global, dir_gat_global, dir_gt_global, dir_gcn_local, dir_gat_local, dir_gt_local]
runs_dirs = [maindir]
runs = sum(([ospj(runs_dir, x) for x in osl(runs_dir)] for runs_dir in runs_dirs), [])
tot_steps_len = 2300#75#150#75#200
runs = natural_sort(runs)

['fcscout_210h-10hp-400bs-1em4lr-eval_NOOPT_', 'gcnscout_210h-10hp-400bs-1em4lr-eval_NOOPT_global', 'gatscout_210h-10hp-400bs-1em4lr-eval_NOOPT_global', 'gtscout_210h-10hp-400bs-1em4lr-eval_NOOPT_global']


In [5]:
all_filters_runs = [
    [run for run in runs if runs_filter in run]
    for runs_filter in runs_filters
]

In [7]:
print(all_filters_runs)

[['/home/vchad/ray_results/fcscout_210h-10hp-400bs-1em4lr-eval_NOOPT_SEED0_g13ve0g9', '/home/vchad/ray_results/fcscout_210h-10hp-400bs-1em4lr-eval_NOOPT_SEED1_5bzlyb1x', '/home/vchad/ray_results/fcscout_210h-10hp-400bs-1em4lr-eval_NOOPT_SEED2_ycpnd43r', '/home/vchad/ray_results/fcscout_210h-10hp-400bs-1em4lr-eval_NOOPT_SEED3_5alnt88y', '/home/vchad/ray_results/fcscout_210h-10hp-400bs-1em4lr-eval_NOOPT_SEED4_14dd6bc8', '/home/vchad/ray_results/fcscout_210h-10hp-400bs-1em4lr-eval_NOOPT_SEED5__3zqjrwu', '/home/vchad/ray_results/fcscout_210h-10hp-400bs-1em4lr-eval_NOOPT_SEED6_zr54fm0q', '/home/vchad/ray_results/fcscout_210h-10hp-400bs-1em4lr-eval_NOOPT_SEED7_xx5ryash', '/home/vchad/ray_results/fcscout_210h-10hp-400bs-1em4lr-eval_NOOPT_SEED8_2_m3ao_4', '/home/vchad/ray_results/fcscout_210h-10hp-400bs-1em4lr-eval_NOOPT_SEED9_9d4j1n_w'], ['/home/vchad/ray_results/gcnscout_210h-10hp-400bs-1em4lr-eval_NOOPT_global_SEED0_b5x9jyti', '/home/vchad/ray_results/gcnscout_210h-10hp-400bs-1em4lr-eval_NO

# Calculate p-values on a per-starting location basis

In [8]:
# Check out runs on a per-start-point basis
import json
def get_best_eval_rewards(run):
    # Fix json file because for some reason it's multiple documents in one
    run_json_data_file = "result.json"
    with open(os.path.join(run, run_json_data_file), "r") as file:
        lines = file.readlines()
    results = [json.loads(result) for result in lines]

    # Get the highest eval rewards and return
    results_ordered_by_eval = sorted([(x["evaluation"]["episode_reward_mean"], x) for x in results], key=lambda x: x[0], reverse=True)
    best_eval, best_result_by_eval = results_ordered_by_eval[0]
    best_result_by_eval_rewards = best_result_by_eval["evaluation"]["hist_stats"]["episode_reward"]
    return best_result_by_eval_rewards

# all_runs_data shape: (n_experiments_to_compare, n_runs_per_experiment, n_evals_per_run/starting_points)
all_runs_data = []
for filter_runs in all_filters_runs:
    curr_run_data = []
    for run in filter_runs:
        run_best_eval_data = get_best_eval_rewards(run)
        curr_run_data.append(run_best_eval_data)
    all_runs_data.append(curr_run_data)

# get stats for mapped values for a given experiment
def get_stats_by_start(all_runs_by_start):
    stats = {}
    baselines = all_runs_by_start[0]
    for exp_name, experiment in zip(runs_filters, all_runs_by_start):
        stats[exp_name] = {}
        for starting_pt_idx in range(len(experiment)):
            ci95z = 1.96
            ci90z = 1.645
            alternative = "less" # "two-sided" #
            baselines_idx = baselines[starting_pt_idx]
            experiment_idx = experiment[starting_pt_idx]
            ttest_results = ttest_ind(baselines_idx, experiment_idx, alternative=alternative, equal_var=False)
            mu, sigma = torch.mean(experiment_idx), torch.std(experiment_idx)
            stats[exp_name][starting_pt_idx] = {
                "str": f"{mu}+/-{ci90z*sigma} -- p={ttest_results.pvalue}",
            }
    return stats

import torch
import pprint
all_runs_by_starting_point = torch.Tensor(all_runs_data).permute(0, 2, 1) # (n_experiments, starting_point, n_runs)
results_by_starting_pt = get_stats_by_start(all_runs_by_starting_point)
printer = pprint.PrettyPrinter(width=160)
printer.pprint(results_by_starting_pt)

{'fcscout_210h-10hp-400bs-1em4lr-eval_NOOPT_': {0: {'str': '312.8999938964844+/-341.5931396484375 -- p=0.5'},
                                                1: {'str': '361.29998779296875+/-242.04931640625 -- p=0.5'},
                                                2: {'str': '345.3999938964844+/-302.3631591796875 -- p=0.5'},
                                                3: {'str': '365.5+/-247.70741271972656 -- p=0.5'},
                                                4: {'str': '342.6000061035156+/-306.9483337402344 -- p=0.5'},
                                                5: {'str': '312.79998779296875+/-341.4252624511719 -- p=0.5'},
                                                6: {'str': '286.20001220703125+/-331.5061950683594 -- p=0.5'},
                                                7: {'str': '312.20001220703125+/-342.3741455078125 -- p=0.5'},
                                                8: {'str': '262.20001220703125+/-361.77435302734375 -- p=0.5'},
                 

# Calculate p-values on a wholistic basis (v1)

In [9]:
# get stats for mapped values for a given experiment
def get_stats_all(all_runs_by_start):
    all_runs_data = all_runs_by_start.reshape(all_runs_by_start.shape[0], -1)
    stats = {}
    baselines = all_runs_data[0]
    for exp_name, vals in zip(runs_filters, all_runs_data):
        mu = torch.mean(vals)
        sigma = torch.std(vals)
        ci95z = 1.96
        ci90z = 1.645
        alternative = "less" # "two-sided"
        ttest_results = ttest_ind(baselines.numpy(), vals.numpy(), alternative=alternative, equal_var=False)
        stats[exp_name] = {
            "str": f"{mu}+/-{ci90z*sigma} -- p={ttest_results.pvalue}",
            "latex": f"{mu} $\pm$ {ci95z*sigma} & {ttest_results.pvalue:1.3f}"
        }
    return stats
results_by_starting_pt = get_stats_all(all_runs_by_starting_point)
printer = pprint.PrettyPrinter(width=160)
printer.pprint(results_by_starting_pt)

{'fcscout_210h-10hp-400bs-1em4lr-eval_NOOPT_': {'latex': '328.21112060546875 $\\pm$ 351.88555908203125 & 0.500',
                                                'str': '328.21112060546875+/-295.33251953125 -- p=0.5'},
 'gatscout_210h-10hp-400bs-1em4lr-eval_NOOPT_global': {'latex': '351.6555480957031 $\\pm$ 323.3451843261719 & 0.099',
                                                       'str': '351.6555480957031+/-271.3789978027344 -- p=0.09893562995921536'},
 'gcnscout_210h-10hp-400bs-1em4lr-eval_NOOPT_global': {'latex': '281.022216796875 $\\pm$ 374.0275573730469 & 0.992',
                                                       'str': '281.022216796875+/-313.9159851074219 -- p=0.991910853395269'},
 'gtscout_210h-10hp-400bs-1em4lr-eval_NOOPT_global': {'latex': '257.8833312988281 $\\pm$ 280.2969970703125 & 1.000',
                                                      'str': '257.8833312988281+/-235.24925231933594 -- p=0.9999752748270376'}}


# Calculate p-values on a wholistic basis (v2)

In [10]:
run_data_file = "progress.csv"
#run_data_col = "episode_reward_mean"
run_data_col = "evaluation/episode_reward_mean"
# run_data_col = "ray/tune/episode_reward_mean"
run_step_col = "timesteps_total"
all_runs_data = []
for filter_runs in all_filters_runs:
    curr_run_data = []
    for run in filter_runs:
        #parse_exp_json(run) # TODO
        datafile = pd.read_csv(ospj(run, run_data_file))
        curr_run_data.append(datafile[run_data_col])
    all_runs_data.append(curr_run_data)
runs_steps = [
    pd.read_csv(ospj(run, run_data_file))[run_step_col]
    for run in runs
]

### given data, run "map" on data to get values suitable for analysis

In [11]:
# finds the first training step when reward exceeds a given value ge
def first_ep_to_val(runs, steps, ge=30):
    first_eps = []
    for run, steps in zip(runs, steps):
        found = False
        for step, val in zip(steps, run):
            if val >= ge:
                first_eps.append(step)
                found = True
                break
        if not found:
            first_eps.append(-1)
    return first_eps

# finds max avg reward
ZONE1_SPAWNS = [85, 87, 94, 95, 96, 102, 103, 111, 112]
ZONE2_SPAWNS = [98, 99, 106, 106, 107, 108, 113, 114, 115]
POSSIBLE_SPAWNS = ZONE1_SPAWNS + ZONE2_SPAWNS
def max_avg_reward(runs, steps, N_EVAL_EPISODES = len(POSSIBLE_SPAWNS), n_top=-1):
    max_vals = []
    for run in runs:
        if len(run) < tot_steps_len: continue
        #max_vals += [max(run)]*N_EVAL_EPISODES
        max_vals.append(max(run))
    if n_top > 0:
        max_vals = sorted(max_vals)[::-1][:n_top]
    max_vals *= N_EVAL_EPISODES
    return max_vals

# returns last reward
def last_ep_reward(runs, steps):
    last_vals = []
    for run in runs:
        if len(run) < tot_steps_len: continue
        last_vals.append(run.tolist()[-1])
    return last_vals

In [12]:
first_ep_to_30 = [
    first_ep_to_val(filter_runs_data, runs_steps)
    for filter_runs_data in all_runs_data
]
max_reward = [
    max_avg_reward(filter_runs_data, runs_steps, n_top=5)
    for filter_runs_data in all_runs_data
]
last_reward = [
    last_ep_reward(filter_runs_data, runs_steps)
    for filter_runs_data in all_runs_data
]

### given "map" values, run "reduce" to aggregate values and show significance

In [13]:
# pretty print a dictionary
def pretty_print(d, tabs=0, tabsize=4):
    for k in d:
        v = d[k]
        nspaces = " " * tabsize * tabs
        if type(v) == dict:
            print(f"{nspaces}{k}:")
            pretty_print(v, tabs+1, tabsize)
        else:
            print(f"{nspaces}{k}: {v}")

# get stats for mapped values for a given experiment
def get_stats(all_runs_vals, firstn=-1):
    stats = {}
    #baselines = all_runs_vals[0]
    baselines = all_runs_vals[0]
    for filter, vals in zip(runs_filters, all_runs_vals):
        if firstn > 0: vals = vals[:firstn]
        mu = np.mean(vals)
        sigma = np.std(vals)
        ci95z = 1.96
        ci90z = 1.645
        alternative = "less" # "two-sided" #
        ttest_results = ttest_ind(baselines, vals, alternative=alternative, equal_var=False)
        stats[filter] = {
            "str": f"{mu}+/-{ci90z*sigma} -- p={ttest_results.pvalue}",
            "latex_per_agent": f"{mu/2:.3f} $\pm$ {ci95z*sigma/2:0.3f} & {ttest_results.pvalue:1.3f}",
            #"median": np.median(vals),
            #"std": sigma,
            #"ci": [mu-ci95z*sigma, mu+ci95z*sigma],
            "n": len(vals),
            "vals": vals[::len(POSSIBLE_SPAWNS)],
        }
    return stats

In [14]:
firstn = -1
#first_ep_to_30_stats = get_stats(first_ep_to_30, firstn=firstn)
max_reward_stats = get_stats(max_reward, firstn=firstn)
last_reward_stats = get_stats(last_reward, firstn=firstn)

In [15]:
def print_stat(stats, name=""):
    assert name != ""
    print(name+" stats:")
    pretty_print(stats)
    print()

print_stat(max_reward_stats, "max reward")
#print_stat(first_ep_to_30_stats, "first ep to 30")
print_stat(last_reward_stats, "last reward")


max reward stats:
fcscout_210h-10hp-400bs-1em4lr-eval_NOOPT_:
    str: 464.4555555555556+/-2.4175081893716537 -- p=0.5
    latex_per_agent: 232.228 $\pm$ 1.440 & 0.500
    n: 90
    vals: [466.5, 463.55555555555554, 465.72222222222223, 462.44444444444446, 464.05555555555554]
gcnscout_210h-10hp-400bs-1em4lr-eval_NOOPT_global:
    str: 431.69999999999993+/-90.50762825800315 -- p=0.9999998896319878
    latex_per_agent: 215.850 $\pm$ 53.919 & 1.000
    n: 90
    vals: [465.72222222222223, 438.1666666666667, 465.5, 323.72222222222223, 465.3888888888889]
gatscout_210h-10hp-400bs-1em4lr-eval_NOOPT_global:
    str: 464.73333333333335+/-1.3936730646428708 -- p=0.06230363748421467
    latex_per_agent: 232.367 $\pm$ 0.830 & 0.062
    n: 90
    vals: [466.3333333333333, 464.22222222222223, 464.72222222222223, 463.8888888888889, 464.5]
gtscout_210h-10hp-400bs-1em4lr-eval_NOOPT_global:
    str: 337.93333333333334+/-154.27790747597876 -- p=1.0
    latex_per_agent: 168.967 $\pm$ 91.910 & 1.000
    n: 